In [1]:
%%time
# import necessary libraries
from sklearn.preprocessing import Imputer
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
# from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# read data from file
train_set = pd.read_csv("train-data.txt")
test_set = pd.read_csv("testx.csv")
# print(train_set.shape,test_set.shape)

# add columns for data set
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels[:-1]

# combine two data set
def id(a):
    for i in range(14):
        if a == col_labels[i]:
            return i
combined_set = pd.concat([train_set, test_set], axis = 0,sort=False)
isMissing = np.zeros(combined_set.shape) # mark the missing data location
cnum = 0
for col in col_labels:
    if col == 'wage_class':
        break
    t = list(combined_set[col])
    for i in range(38560):
        if t[i] == ' ?':
            isMissing[i,cnum] = 1
    cnum += 1
combined_set = combined_set.replace(' ?',np.nan)

sum(sum(isMissing))

# mark the reflection of wage_class
list(combined_set['wage_class'])[0]

# replace strings with integers and 
# mark the way to fix data set 1 means using mode while 0 means using means
typ = [0 for i in range(14)]
cnt = 0
for feature in combined_set.columns:
    if combined_set[feature].dtype == 'object':
        typ[cnt] = 1
        cnt += 1
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes

# mark the reflection of wage_class
list(combined_set['wage_class'])[0]

# test
s = set([])
cnum = 0
for col in col_labels:
    if col == 'wage_class':
        break
    t = list(combined_set[col])
    for i in range(38560):
        if isMissing[i,cnum] > 0:
            s.update([t[i]])
    if len(s) > 1:
        break
    cnum += 1
s

# repleace -1 with np.nan
cnum = 0
cnt = 0
for col in col_labels:
    if col == 'wage_class':
        break
    t = list(combined_set[col])
    for i in range(38560):
        if isMissing[i,cnum] > 0:
            t[i] = np.nan
            cnt += 1
    combined_set[col] = t
    cnum += 1
# print(cnt)

# fix the data set
for i in range(14):
    if typ[i] == 1:
        combined_set[col_labels[i]]=combined_set[col_labels[i]].fillna(combined_set[col_labels[i]].mode().iloc[0])
    else:
        combined_set[col_labels[i]]=combined_set[col_labels[i]].fillna(combined_set[col_labels[i]].mean())

CPU times: user 2.01 s, sys: 92 ms, total: 2.1 s
Wall time: 2.15 s


In [2]:
train = combined_set[:32560]
p_train = train[train['wage_class'] > 0]
n_train = train[train['wage_class'] == 0]
print(p_train.shape,n_train.shape)
#train_x = combined_set.drop(columns=['wage_class'])[:32560]
#train_y = combined_set.drop(columns = col_labels[:-1])[:32560]
test_x = combined_set.drop(columns=['wage_class'])[32560:]
# print(train_x.shape,train_y.shape,test_x.shape)

# data process done
x = train.iloc[:, :-1] # 切片，得到输入x
y = train.iloc[:, -1] # 切片，得到标签y
model_smote = SMOTE(sampling_strategy=0.4) # 建立SMOTE模型对象
x_smote_resampled, y_smote_resampled = model_smote.fit_sample(x,y) # 输入数据并作过抽样处理
x_smote_resampled = pd.DataFrame(x_smote_resampled, columns=col_labels[:-1]) # 将数据转换为数据框并命名列名
y_smote_resampled = pd.DataFrame(y_smote_resampled,columns=['wage_class']) # 将数据转换为数据框并命名列名
smote_resampled = pd.concat([x_smote_resampled, y_smote_resampled],axis=1) # 按列合并数据框
groupby_data_smote = smote_resampled.groupby('wage_class').count() # 对label做分类汇总
#print (groupby_data_smote) # 打印输出经过SMOTE处理后的数据集样本分类分布
#print(x_smote_resampled.shape,y_smote_resampled.shape)
# split the data set
#train_x = combined_set.drop(columns=['wage_class'])[:32560]
#train_y = combined_set.drop(columns = col_labels[:-1])[:32560]
#test_x = combined_set.drop(columns=['wage_class'])[32560:]
# print(train_x.shape,train_y.shape,test_x.shape)
train_x = x_smote_resampled
train_y = y_smote_resampled
print(train_x.shape)

(7841, 15) (24719, 15)
(34606, 14)


In [3]:
#boostingTrain_x = pd.DataFrame([],columns=col_labels[:-1],dtype='int64')
#boostingTrain_y = pd.DataFrame([],columns=col_labels[14:],dtype='int64')
#train_x['sex'] > 0

In [4]:
sumans = np.zeros(6000,dtype = 'int64')
arr = np.array([[2, 1, 2, 2, 0, 1, 2],
    [2, 2, 0, 0, 1, 0, 0],
    [1, 0, 2, 1, 0, 2, 1],
    [2, 0, 2, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [2, 1, 2, 2, 0, 1, 2],
    [2, 2, 0, 0, 1, 0, 0],
    [1, 0, 2, 1, 0, 2, 1],
    [2, 0, 2, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 0, 0]])
def fun(par,num_rounds):
    # generate random train set and valid set
    X_train,X_valid,y_train,y_valid = train_test_split(
        train_x, train_y, test_size=0.3)
    Dtrain=xgb.DMatrix(X_train,y_train)

    # train model
    plst=par.items()
    model=xgb.train(plst,Dtrain,num_rounds)

    # predict valid data set
    dvalid = xgb.DMatrix(X_valid)
    ans = model.predict(dvalid)
    y_valid1 = list(y_valid['wage_class'])
    dvalid = xgb.DMatrix(X_valid)
    ans = model.predict(dvalid)

    # calculate the F1 score
    length = len(y_valid)
    tp = tn = fp = fn = cnt = 0
    for i in range(length):
        p,t = ans[i],y_valid1[i]
        if p == 1 and t == 1:
            tp += 1
        elif p == 0 and t == 0:
            tn += 1
        elif p == 1 and t == 0:
            fp += 1
        elif p == 0 and t == 1:
            fn += 1
    pp = tp / (tp + fp)
    rr = tp / (tp + fn)
    f1 = 2 * (pp * rr) / (pp + rr)
    test = xgb.DMatrix(test_x)
    final = np.array(model.predict(test),dtype='int64')
    global sumans
    sumans = sumans + final
    pos = 0
    for i in range(len(final)):
        if final[i] == 1:
            pos += 1
    #print(pos / len(final))
    final = pd.DataFrame({'y':final})
    file = pd.concat([final],sort=False)
    file.to_csv("ans.csv",index=False,sep=',')
    # calculate the precision
    #for i in range(length):
    #    if ans[i] == y_valid1[i]:
    #        cnt += 1
    #precision = cnt / length
    # print(precision)

    print(f1)

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 2,
    'gamma': 0.5,# 0 ~ 1 0.5 3
    'max_depth': 5,# 5 ~ 7 3
    'lambda': 1,
    'subsample': 0.8,# 0.6 ~ 0.8 0.1 3
    'colsample_bytree': 0.6,# 0.6 ~ 0.8 0.1 3
    'min_child_weight': 2, # 2 ~ 4 1 3
    'eta': 0.11, # 0.09 ~ 0.11 0.01 3
#    'seed': 1000,
#    nthread : 4
    'silent': 1,
}

delta = {}
delta['gamma'] = np.linspace(0,1,num=3)
delta['max_depth'] = np.linspace(5,7,num=3,dtype='int64')
delta['subsample'] = np.linspace(0.6,0.8,num=3)
delta['colsample_bytree'] = np.linspace(0.6,0.8,num=3)
delta['min_child_weight'] = np.linspace(2,4,num=3,dtype='int64')
delta['eta'] = np.linspace(0.09,0.11,num=3)
delta['num_rounds'] = np.linspace(190,210,num=3,dtype='int64')

for i in range(1):
    [l,m,n,o,p,q,r] = arr[i]
    params['gamma'] = delta['gamma'][l]
    params['max_depth'] = delta['max_depth'][m]
    params['subsample'] = delta['subsample'][n]
    params['colsample_bytree'] = delta['colsample_bytree'][o]
    params['min_child_weight'] = delta['min_child_weight'][p]
    params['eta'] = delta['eta'][q]
    fun(params,delta['num_rounds'][r])

sumans = pd.DataFrame({'y':sumans})
file = pd.concat([sumans],sort=False)
file.to_csv("ans.csv",index=False,sep=',')

0.7722664735698769
